In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import time

In [ ]:
# Obtenir la date actuelle au format 'AAAA-MM-JJ'
date_actuelle = time.strftime('%Y-%m-%d')

# Récupérer les données jusqu'à la date actuelle pour le prix du Bitcoin en USD
btc_data = yf.download('BTC-USD', end=date_actuelle)

# Ajouter une colonne de dates au DataFrame
btc_data['Date'] = btc_data.index

# Réorganiser les colonnes pour avoir 'Date' en première position
btc_data = btc_data[['Date'] + [col for col in btc_data.columns if col != 'Date']]

# Enregistrez les données mises à jour dans le fichier CSV en écrasant toutes les données existantes
btc_data.to_csv('btc_price_data.csv', index=False, mode='w')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Charger les données depuis le fichier CSV
data = pd.read_csv('btc_price_data.csv')

# Sélectionner les caractéristiques (prix actuels)
prix_actuels = data["Open"].values.reshape(-1, 1)

# Décaler les prix actuels pour obtenir les prix futurs (décalage d'une ligne)
prix_futurs = np.roll(prix_actuels, shift=-1)

# Supprimer la dernière ligne pour égaliser la forme des caractéristiques et des étiquettes
prix_actuels = prix_actuels[:-1]
prix_futurs = prix_futurs[:-1]

# Normaliser les caractéristiques et les étiquettes
scaler = MinMaxScaler()
prix_actuels_normalisés = scaler.fit_transform(prix_actuels)
prix_futurs_normalisés = scaler.transform(prix_futurs)


In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(256,activation ="relu"))
model.add(tf.keras.layers.Dense(128,activation ="relu"))
model.add(tf.keras.layers.Dense(50,activation ="relu"))
model.add(tf.keras.layers.Dense(1,activation ="linear"))



In [ ]:
model.compile(
    loss ="mean_squared_error",
    optimizer = "adam",
    metrics = ["mae"]
)

In [ ]:
# Entraîner le modèle avec les prix actuels et les prix futurs
history = model.fit(prix_actuels_normalisés, prix_futurs_normalisés, epochs=100, validation_split=0.2)


In [ ]:
# Sélectionner les caractéristiques pour la prédiction
caractéristiques = prix_actuels[-1].reshape(1, -1)  # Utilisez le dernier prix actuel

# Normaliser les caractéristiques
caractéristiques_normalisées = scaler.transform(caractéristiques)

# Faire la prédiction
prix_prédits_normalisés = model.predict(caractéristiques_normalisées)

# Inverser la normalisation pour obtenir le prix prédit en valeur réelle
prix_prédits = scaler.inverse_transform(prix_prédits_normalisés)

f = prix_prédits[0][0]

print(f"le prix prévu pour environ demain est d'environ : {f} USD")


In [ ]:
loss_curve = history.history["loss"]
acc_curve = history.history["mae"]
loss_val_curve = history.history["val_loss"]
acc_val_curve = history.history["val_mae"]

plt.plot(loss_curve, label="Train")
plt.plot(loss_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.plot(acc_curve, label="Train")
plt.plot(acc_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("mae")
plt.show()